In [ ]:
!pip install tensorflow==1.14

     |████████████████████████████████| 109.2MB 84kB/s 
     |████████████████████████████████| 51kB 4.5MB/s 
     |████████████████████████████████| 491kB 30.0MB/s 
     |████████████████████████████████| 3.2MB 41.9MB/s 
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tensorflow-estimator-2.4.0
  Found existing installation: tensorboard 2.4.1
    Uninstalling tensorboard-2.4.1:
      Successfully uninstalled tensorboard-2.4.1
  Found existing installation: tensorflow 2.4.1
    Uninstalling tensorflow-2.4.1:
      Successfully uninstalled tensorflow-2.4.1


In [ ]:
import tensorflow as tf
import tensorflow.contrib.slim as slim
import numpy as np

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

## The Bandit

손잡이가 4개인 밴딧을 만들어보자.  
pullBandit : 랜덤한 포준정규분포 값을 하나 생성하고 인수로 받은 수보다 크면 1, 아니면 -1을 반환.   
따라서 인수로 넘겨지는 수가 작을수록 (+)의 보상이 돌아올 가능성이 높음.  
에이전트가 언제나 (+)의 보상을 가져올 손잡이를 선택하도록 학습하는 것이 목표.

In [ ]:
# 밴딧의 손잡이 목록을 작성한다.
# 현재 손잡이 4가 가장 자주 양의 보상을 제공하도록 설정되어 있다.
bandit_arms = [0.2, 0, -0.2, -2]
num_arms = len(bandit_arms)

def pullBandit(bandit) :
  # 랜덤한 값을 구한다
  result = np.random.randn(1)
  if result > bandit :
    # 양의 보상을 반환
    return 1
  else :
    # 음의 보상을 반환
    return -1

## 에이전트

간단한 신경망을 구현해보자.  
각 밴딧 손잡이에 대한 일련의 값들로 구성되어 있고, 여기서 각 값은 해당 밴딧을 선택할 때 반환되는 보상의 추정값을 의미함.  
policy gradient를 이용해 선택된 액션에 대한 보상이 큰 쪽으로 이동해가며 에이전트 업데이트

In [ ]:
tf.reset_default_graph()

# 네트워크의 feedforward 부분 구현
weights = tf.Variable(tf.ones([num_arms]))
output = tf.nn.softmax(weights)

In [ ]:
# 학습 과정을 구현한다.
# 보상과 선택된 액션을 네트워크에 피드해줌으로써 비용을 계산하고
# 비용을 이용해 네트워크 업데이트
reward_holder = tf.placeholder(shape=[1], dtype=tf.float32) # placeholder : 텐서를 만들긴 하지만 값을 넣진않음(미리 생성 후 값은 나중에 넣기 위해)
action_holder = tf.placeholder(shape=[1], dtype=tf.int32)

responsible_output = tf.slice(output, action_holder, [1])
loss = -(tf.log(responsible_output)*reward_holder)
optimizer = tf.train.AdamOptimizer(learning_rate=1e-3)
update = optimizer.minimize(loss)

## 에이전트 학습시키기

환경 내에서 에이전트는 액션을 취함으로써 학습하고 보상을 받음.  
보상과 액션을 이용하여 시간의 경과에 따라 최고의 보상을 받게 될 액션을 더 자주 선택하려면 네트워크를 어떻게 업데이트해나갈지에 대해 알 수 있음.

In [ ]:
# 에이전트를 학습시킬 총 에피소드의 수를 설정
total_episodes = 1000
# 밴딧 손잡이에 대한 점수판을 0으로 설정
total_reward = np.zeros(num_arms)

init = tf.global_variables_initializer()

# 텐서플로 그래프를 론칭
with tf.Session() as sess :
  sess.run(init)
  i = 0
  while i < total_episodes :
    # 볼츠만 분포에 따라 액션 선택
    actions = sess.run(output)
    a = np.random.choice(actions, p = actions)
    action = np.argmax(actions == a)

    # 밴딧 손잡이 중 하나를 선택하여 보상 받기
    reward = pullBandit(bandit_arms[action])

    # 네트워크 업데이트
    _, resp, ww = sess.run([update, responsible_output, weights],
                           feed_dict = {reward_holder:[reward], action_holder:[action]})
    
    # 보상의 총계 업데이트
    total_reward[action] += reward
    if i % 50 == 0 :
      print("Running reward for the " + str(num_arms) + " arms of the bandit : " + str(total_reward))
    i += 1

print("\nThe agent thinks arm " + str(np.argmax(ww) + 1) + " is the most promising.....")
if np.argmax(ww) == np.argmax(-np.array(bandit_arms)) :
  print("...and it was right!")
else :
  print("...and it was wrong!")

Running reward for the 4 arms of the bandit : [1. 0. 0. 0.]
Running reward for the 4 arms of the bandit : [ 6. -5.  2. 12.]
Running reward for the 4 arms of the bandit : [ 3. -7.  7. 26.]
Running reward for the 4 arms of the bandit : [-2. -8. 12. 35.]
Running reward for the 4 arms of the bandit : [  0. -14.  10.  47.]
Running reward for the 4 arms of the bandit : [ -2. -12.  15.  62.]
Running reward for the 4 arms of the bandit : [-6. -5. 10. 74.]
Running reward for the 4 arms of the bandit : [-12.  -2.  14.  91.]
Running reward for the 4 arms of the bandit : [-12.   0.  16. 107.]
Running reward for the 4 arms of the bandit : [-13.   2.  23. 123.]
Running reward for the 4 arms of the bandit : [-17.   7.  30. 141.]
Running reward for the 4 arms of the bandit : [-22.   4.  29. 158.]
Running reward for the 4 arms of the bandit : [-24.   6.  27. 172.]
Running reward for the 4 arms of the bandit : [-29.   6.  27. 187.]
Running reward for the 4 arms of the bandit : [-25.   9.  35. 206.]
Runn